## Directory parameters

In [1]:
# Update working directory

%cd /home/hugo/Bureau/PoincareMSA/

/home/hugo/Bureau/PoincareMSA


/home/hugo/miniconda3/envs/poincare/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import sys

# Add the project root to Python path
project_root = "/home/hugo/Bureau/PoincareMSA"
if project_root not in sys.path:
    sys.path.append(project_root)

## Librairies

In [3]:
import os
import itertools
import pandas as pd
import torch   # ⬅️ nécessaire pour charger les embeddings AAE (.pt)

# Fonctions internes déjà existantes
from scripts.visualize_projection.pplots_new import (
    read_embeddings,
    plot_embedding_interactive
)

from scripts.visualize_projection.pplots_new import read_embeddings, plot_embedding_interactive
from scripts.build_poincare_map.poincare_maps import PoincareMaps
from scripts.build_poincare_map.embedding_quality_score import get_quality_metrics
from sklearn.cluster import AgglomerativeClustering, SpectralClustering
from sklearn.metrics import adjusted_rand_score, fowlkes_mallows_score
from scipy import stats
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


## Parameters

In [4]:
data_type = "plm"       # "plm", "plm_aae", "pssm", "RFA_matrix"

# Annotation CSV
path_annotation_csv = "examples/globins/globin_colors_new.csv"

# Embeddings or fasta folder depending on type
embedding_paths = {
    "plm":       "embeddings/ankh_base_globins/",
    "plm_aae":   "embeddings/aae_embeddings/ankh_base_globins/",
    "pssm":      "examples/globins/glob.mfasta",
    "RFA_matrix": None
}

# Output bases
output_base = "grid_search_results/" + data_type
os.makedirs(output_base, exist_ok=True)

In [5]:
grid = {
    "knn":       [5],
    "gamma":     [0.5],
    "sigma":     [1],
    "cospca":    [0],
    "epochs":    [50],
    "seed":      [2, 4],
    "distance":  ["cosine"],
}

## Usefull functions

In [6]:
def get_scores(true_labels, pred_labels):
    ARI = adjusted_rand_score(true_labels, pred_labels)
    FMS = fowlkes_mallows_score(true_labels, pred_labels)
    return ARI, FMS

def detect_cluster(distances, n_clusters=2, clustering_name='agglomerative'):
    if clustering_name == 'agglomerative':
        clustering = AgglomerativeClustering(n_clusters=n_clusters, linkage='average').fit(distances)
        labels = clustering.labels_
    return labels


In [7]:
def build_command(params, folder_output):
    if data_type in ["plm", "plm_aae"]:
        input_path = embedding_paths[data_type]
        plm_flag = "True"
        mid_output = folder_output + "/matrices/"
        os.makedirs(mid_output, exist_ok=True)

    elif data_type == "pssm":
        input_path = "globins_data/with_mfasta/"
        plm_flag = "False"
        mid_output = input_path

    elif data_type == "RFA_matrix":
        input_path = "None"
        plm_flag = "True"
        mid_output = "RFA_matrix/globins/with_plm_embeddings/"

    cmd = (
        f"PYTHONPATH={project_root}:$PYTHONPATH python scripts/build_poincare_map/main.py "
        f"--input_path {input_path} "
        f"--output_path {folder_output} "
        f"--plm_embedding {plm_flag} "
        f"--matrices_output_path {mid_output} "
        f"--distlocal {params['distance']} "
        f"--gamma {params['gamma']} "
        f"--pca {params['cospca']} "
        f"--epochs {params['epochs']} "
        f"--seed {params['seed']} "
        f"--knn {params['knn']}"
    )
    return cmd


def load_projection(folder_output, params):
    proj_file = (
        f"{folder_output}/PM{params['knn']}sigma={params['sigma']:.2f}"
        f"gamma={params['gamma']:.2f}{params['distance']}pca={params['cospca']}"
        f"_seed{params['seed']}.csv"
    )
    return proj_file

In [ ]:
# Comparison function
def compare_projections(results):
    df_results = pd.DataFrame(results)

    # Convert params to string for visualization
    df_results["params_str"] = df_results["params"].apply(lambda x: "_".join([f"{k}{v}" for k, v in x.items()]))

    # Print available metrics
    print("\n=== Comparison of Projections ===")
    available_cols = ["params_str", "ARI", "FMS"]
    if "Qlocal" in df_results.columns and df_results["Qlocal"].notna().any():
        available_cols.append("Qlocal")
    if "Qglobal" in df_results.columns and df_results["Qglobal"].notna().any():
        available_cols.append("Qglobal")
    print(df_results[available_cols])

    # Plot ARI if available
    if "ARI" in df_results.columns and df_results["ARI"].notna().any():
        fig_ari = px.strip(df_results, x="params_str", y="ARI", title="ARI by parameter combination")
        fig_ari.update_layout(xaxis_title="Parameters", yaxis_title="ARI")
        fig_ari.show()

    # Plot FMS if available
    if "FMS" in df_results.columns and df_results["FMS"].notna().any():
        fig_fms = px.strip(df_results, x="params_str", y="FMS", title="FMS by parameter combination")
        fig_fms.update_layout(xaxis_title="Parameters", yaxis_title="FMS")
        fig_fms.show()

    # Plot Qnx if available
    if "Qnx" in df_results.columns and any(len(qnx) > 0 for qnx in df_results["Qnx"]):
        meth_palette = dict(zip(
            df_results["params_str"].unique(),
            sns.color_palette(palette=None, n_colors=len(df_results["params_str"].unique())).as_hex()
        ))

        plt.figure(figsize=(12, 6))
        for i, row in df_results.iterrows():
            if isinstance(row["Qnx"], (list, np.ndarray)) and len(row["Qnx"]) > 0:
                sns.lineplot(
                    x=range(1, len(row["Qnx"]) + 1),
                    y=row["Qnx"],
                    color=meth_palette[row["params_str"]],
                    label=row["params_str"]
                )

        plt.xlabel("K")
        plt.ylabel("Qnx")
        plt.ylim([0, 1.1])
        plt.title("Qnx by parameter combination")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.show()

## Grid search & plot

In [ ]:
# %%
print("=== Starting grid search ===")
df_annotation = pd.read_csv(path_annotation_csv)

# List to store the results of projections
results = []

# Define a custom color palette for visualization
globin_palette = {
    'Echinodermata': '#086b75', 'Arthropoda': '#0b237c', 'Mollusca': '#512ff8',
    'Annelida': '#a191f3', 'Chordata': '#26c9d9', 'Cnidaria': '#ad288b',
    'Porifera': '#fdb7fd', 'Placozoa': '#e9bd6b', 'Bacteria': '#f10000',
    'Nematoda': '#5d78e3', 'Hemichordata': '#b0ffe8', 'Fungi': '#a0e361',
    'Viridiplantae': '#4d9b03'
}

# Generate all parameter combinations
keys = list(grid.keys())
values = list(grid.values())
combinations = list(itertools.product(*values))

for combo in combinations:
    params = dict(zip(keys, combo))

    # Create a folder for each combination
    folder_name = "_".join([f"{k}{v}" for k, v in params.items()])
    folder_output = os.path.join(output_base, folder_name)
    os.makedirs(folder_output, exist_ok=True)
    print(f"\n--- Running projection for {folder_name} ---")

    # 1. Run Poincaré map
    cmd = build_command(params, folder_output + "/")
    print("CMD:", cmd)
    os.system(cmd)

    # 2. Load projection
    projection_file = load_projection(folder_output, params)
    if not os.path.exists(projection_file):
        print(f"❌ Projection file not found: {projection_file}")
        continue

    df_emb = read_embeddings(projection_file, path_annotation_csv, withroot=False)

    # 3. Generate and display interactive plot for each parameter combination
    if data_type == "plm":
        title = f"Poincaré projection with embeddings: {folder_name}"
    elif data_type == "plm_aae":
        title = f"Poincaré projection with AAE embeddings: {folder_name}"
    elif data_type == "pssm":
        title = f"Poincaré projection without embeddings: {folder_name}"
    else:
        title = f"Poincaré projection with RFA matrix: {folder_name}"

    # Create and display the interactive plot
    fig = plot_embedding_interactive(
        df_emb,
        labels_name="Color_species",
        color_palette=globin_palette,
        title=title,
        fontsize=11
    )
    fig.show()

    # Save the plot to HTML file
    output_html = os.path.join(folder_output, f"projection_{folder_name}.html")
    fig.write_html(output_html)
    print(f"✔ Saved projection to → {output_html}")

    # 4. Calculate quality metrics
    true_labels = df_emb["Color_species"].values
    coord_low = df_emb[["pm1", "pm2"]].values

    # Load original high-dimensional data
    if data_type in ["plm", "plm_aae"]:
        embedding_files = [f for f in os.listdir(embedding_paths[data_type]) if f.endswith('.pt')]
        features = []
        for file in sorted(embedding_files):
            embedding = torch.load(os.path.join(embedding_paths[data_type], file))['embedding']
            features.append(np.mean(embedding, axis=0))
        features = np.array(features)
    elif data_type == "pssm":
        pssm_files = [f for f in os.listdir("examples/globins/") if f.endswith('.aamtx')]
        features = []
        for file in sorted(pssm_files):
            pssm = np.loadtxt(os.path.join("examples/globins/", file)).reshape(-1)
            features.append(pssm)
        features = np.array(features)

    # Calculate all quality metrics
    try:
        model = PoincareMaps(coord_low)
        model.get_distances()
        D_proj = model.distances

        # Detect clusters
        clusters = detect_cluster(D_proj, n_clusters=len(np.unique(true_labels)))
        ARI, FMS = get_scores(true_labels, clusters)

        # Calculate Q metrics
        Qlocal, Qglobal, Kmax, df_Q = get_quality_metrics(
            coord_high=features,
            coord_low=coord_low,
            distance="poincare",
            setting="manifold",
            k_neighbours=5
        )

        # Store all results
        result = {
            "params": params,
            "ARI": ARI,
            "FMS": FMS,
            "Qlocal": Qlocal,
            "Qglobal": Qglobal,
            "Qnx": df_Q.Qnx.to_numpy() if hasattr(df_Q, 'Qnx') else np.zeros(5),
            "projection_file": projection_file,
        }
    except Exception as e:
        print(f"Error calculating metrics: {e}")
        # Store partial results if metrics calculation fails
        result = {
            "params": params,
            "ARI": ARI if 'ARI' in locals() else None,
            "FMS": FMS if 'FMS' in locals() else None,
            "Qlocal": None,
            "Qglobal": None,
            "Qnx": np.zeros(5),
            "projection_file": projection_file,
        }

    results.append(result)

print("\n=== Grid search complete ===")


=== Starting grid search ===

--- Running projection for knn5_gamma0.5_sigma1_cospca0_epochs50_seed2_distancecosine ---
CMD: PYTHONPATH=/home/hugo/Bureau/PoincareMSA:$PYTHONPATH python scripts/build_poincare_map/main.py --input_path embeddings/ankh_base_globins/ --output_path grid_search_results/plm/knn5_gamma0.5_sigma1_cospca0_epochs50_seed2_distancecosine/ --plm_embedding True --matrices_output_path grid_search_results/plm/knn5_gamma0.5_sigma1_cospca0_epochs50_seed2_distancecosine//matrices/ --distlocal cosine --gamma 0.5 --pca 0 --epochs 50 --seed 2 --knn 5
CUDA: True
Random seed set as 2
252 proteins found in folder embeddings/ankh_base_globins/.
['230.pt', '53.pt', '6.pt', '150.pt', '236.pt', '188.pt', '235.pt', '21.pt', '117.pt', '71.pt', '193.pt', '52.pt', '95.pt', '3.pt', '103.pt', '239.pt', '240.pt', '104.pt', '175.pt', '159.pt', '158.pt', '141.pt', '155.pt', '197.pt', '163.pt', '8.pt', '245.pt', '67.pt', '37.pt', '26.pt', '142.pt', '14.pt', '94.pt', '78.pt', '125.pt', '56.pt'

loss: 2.01672: 100%|████████████████████████████| 50/50 [00:03<00:00, 12.91it/s]
Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.
Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


PM computed in 3.87 sec

loss = 2.017e+00
time = 0.071 min


✔ Saved projection to → grid_search_results/plm/knn5_gamma0.5_sigma1_cospca0_epochs50_seed2_distancecosine/projection_knn5_gamma0.5_sigma1_cospca0_epochs50_seed2_distancecosine.html

--- Running projection for knn5_gamma0.5_sigma1_cospca0_epochs50_seed4_distancecosine ---
CMD: PYTHONPATH=/home/hugo/Bureau/PoincareMSA:$PYTHONPATH python scripts/build_poincare_map/main.py --input_path embeddings/ankh_base_globins/ --output_path grid_search_results/plm/knn5_gamma0.5_sigma1_cospca0_epochs50_seed4_distancecosine/ --plm_embedding True --matrices_output_path grid_search_results/plm/knn5_gamma0.5_sigma1_cospca0_epochs50_seed4_distancecosine//matrices/ --distlocal cosine --gamma 0.5 --pca 0 --epochs 50 --seed 4 --knn 5
CUDA: True
Random seed set as 4
252 proteins found in folder embeddings/ankh_base_globins/.
['230.pt', '53.pt', '6.pt', '150.pt', '236.pt', '188.pt', '235.pt', '21.pt', '117.pt', '71.pt', '193.pt', '52.pt', '95.pt', '3.pt', '103.pt', '239.pt', '240.pt', '104.pt', '175.pt', '159.p

loss: 2.01958: 100%|████████████████████████████| 50/50 [00:05<00:00,  8.93it/s]


PM computed in 5.60 sec

loss = 2.020e+00
time = 0.100 min


Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.
Ignoring fixed x limits to fulfill fixed data aspect with adjustable data limits.


✔ Saved projection to → grid_search_results/plm/knn5_gamma0.5_sigma1_cospca0_epochs50_seed4_distancecosine/projection_knn5_gamma0.5_sigma1_cospca0_epochs50_seed4_distancecosine.html

=== Grid search complete ===


## Results

In [13]:
# Call comparison function
compare_projections(results)


=== Comparison of Projections ===
                                          params_str       ARI       FMS  \
0  knn5_gamma0.5_sigma1_cospca0_epochs50_seed2_di...  0.162560  0.323779   
1  knn5_gamma0.5_sigma1_cospca0_epochs50_seed4_di...  0.171194  0.332450   

     Qlocal   Qglobal  
0  0.041831  0.503457  
1  0.044065  0.503014  
